In [1]:
import pandas as pd
import os

In [2]:
PATH_TO_DATASET = r'/media/gagandeep/2E92405C92402AA3/Work/Kaggle/Zips/fake-news'

In [3]:
os.listdir(PATH_TO_DATASET)

['submit.csv', 'test.csv', 'train.csv']

In [4]:
data = pd.read_csv(os.path.join(PATH_TO_DATASET, 'train.csv'))
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
# Drop NA values
data = data.dropna()

In [6]:
# Get features and label from the data
X = data.drop('label', axis=1)
y = data['label']
print("Shape of features: {}".format(X.shape))
print("Shape of labels: {}".format(y.shape))

Shape of features: (18285, 4)
Shape of labels: (18285,)


### Our goal is to build a LSTM model - unidirectional and biderectional to classify fake news

In [7]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [9]:
# Set the vocabulary size -> the size of our corpus.
voc_size = 5000

In [10]:
# One hot representation
messages = X.copy()
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [11]:
messages.reset_index(inplace=True)

In [12]:
import nltk
import re
from nltk.corpus import stopwords

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gagandeep/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
# Pre process the data
# The same steps will be used when making predictions
import tqdm
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in tqdm.tqdm(range(0, len(messages))):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)   

100%|██████████| 18285/18285 [00:34<00:00, 527.95it/s]


In [17]:
corpus[:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [25]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr[:10]
#shape of matrix
#18285 x 5000

[[3441, 3451, 556, 3166, 921, 376, 4172, 3582, 3283, 1407],
 [2439, 4354, 2190, 4748, 2442, 819, 4010],
 [1622, 1331, 1470, 4593],
 [1761, 4931, 1912, 66, 4824, 3843],
 [1093, 2442, 1449, 4266, 2549, 2050, 2442, 4496, 778, 4130],
 [363,
  3504,
  1966,
  4425,
  1826,
  1857,
  587,
  2550,
  3020,
  2695,
  2138,
  4846,
  1903,
  130,
  4010],
 [3922, 1190, 2550, 4479, 3366, 3921, 4612, 3061, 678, 3930, 4758],
 [2593, 3166, 4358, 407, 4870, 1255, 1857, 3956, 678, 3930, 4758],
 [2666, 878, 3710, 2162, 3117, 4944, 578, 4516, 1857, 692],
 [771, 4491, 2848, 958, 906, 3998, 2243, 3960]]

In [26]:
# Now to make the length of each sentance uniform, we will define a fixed length and pad all the sentances which are less than the ficed length value
fixed_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=fixed_length) 

In [30]:
embedded_docs[0:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3441,
        3451,  556, 3166,  921,  376, 4172, 3582, 3283, 1407],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 2439, 4354, 2190, 4748, 2442,  819, 4010],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 1622, 1331, 1470, 4593],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 1761, 4931, 1912,   66, 4824, 3843],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1093,
        2442, 1449, 4266, 2549, 2050, 2442, 4496,  778, 4130]],
      dtype=int32)

In [33]:
from tensorflow.keras.layers import Dropout
# Model Creation - Unidirectional
embedding_dim = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_dim, input_length=fixed_length))
model.add(Dropout(0.3))
model.add(LSTM(100, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
import numpy as np
X = np.array(embedded_docs)
y = np.array(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [35]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=64)

Train on 12799 samples, validate on 5486 samples
Epoch 1/15
12799/12799 [==============================] - 12s 932us/sample - loss: 0.3581 - accuracy: 0.8140 - val_loss: 0.2089 - val_accuracy: 0.9103
Epoch 2/15
12799/12799 [==============================] - 8s 644us/sample - loss: 0.1589 - accuracy: 0.9356 - val_loss: 0.2027 - val_accuracy: 0.9167
Epoch 3/15
12799/12799 [==============================] - 7s 524us/sample - loss: 0.1255 - accuracy: 0.9558 - val_loss: 0.2024 - val_accuracy: 0.9171
Epoch 4/15
12799/12799 [==============================] - 6s 504us/sample - loss: 0.0911 - accuracy: 0.9662 - val_loss: 0.2337 - val_accuracy: 0.9200
Epoch 5/15
12799/12799 [==============================] - 6s 492us/sample - loss: 0.0682 - accuracy: 0.9761 - val_loss: 0.2624 - val_accuracy: 0.9156
Epoch 6/15
12799/12799 [==============================] - 6s 490us/sample - loss: 0.0546 - accuracy: 0.9815 - val_loss: 0.2805 - val_accuracy: 0.9167
Epoch 7/15
12799/12799 [==========================

In [36]:
y_pred = model.predict_classes(X_test)
y_pred

array([[1],
       [0],
       [0],
       ...,
       [1],
       [0],
       [1]], dtype=int32)

In [37]:
from sklearn.metrics import confusion_matrix, classification_report

In [41]:
print("-------------Confusion Matrix -----------------")
confusion_matrix(y_test, y_pred)

-------------Confusion Matrix -----------------


array([[2779,  328],
       [ 129, 2250]])

In [42]:
print("-------------Classification Report -----------------")
classification_report(y_test, y_pred)

-------------Classification Report -----------------


'              precision    recall  f1-score   support\n\n           0       0.96      0.89      0.92      3107\n           1       0.87      0.95      0.91      2379\n\n    accuracy                           0.92      5486\n   macro avg       0.91      0.92      0.92      5486\nweighted avg       0.92      0.92      0.92      5486\n'

In [43]:
model.save('LSTM_Uni_dropout.h5')

In [45]:
from tensorflow.keras.layers import Bidirectional
# Model Creation - Biidirectional
embedding_dim = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_dim, input_length=fixed_length))
model.add(Bidirectional(LSTM(100, activation='relu')))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=64)

Train on 12799 samples, validate on 5486 samples
Epoch 1/15
12799/12799 [==============================] - 18s 1ms/sample - loss: 0.3408 - accuracy: 0.8421 - val_loss: 0.2052 - val_accuracy: 0.9114
Epoch 2/15
12799/12799 [==============================] - 10s 812us/sample - loss: 0.1494 - accuracy: 0.9398 - val_loss: 0.2081 - val_accuracy: 0.9154
Epoch 3/15
12799/12799 [==============================] - 11s 864us/sample - loss: 0.1232 - accuracy: 0.9581 - val_loss: 0.2601 - val_accuracy: 0.9120
Epoch 4/15
12799/12799 [==============================] - 10s 807us/sample - loss: 0.0772 - accuracy: 0.9712 - val_loss: 0.9507 - val_accuracy: 0.9036
Epoch 5/15
12799/12799 [==============================] - 9s 717us/sample - loss: 0.0557 - accuracy: 0.9776 - val_loss: 1.4990 - val_accuracy: 0.9008
Epoch 6/15
12799/12799 [==============================] - 9s 725us/sample - loss: 0.0500 - accuracy: 0.9801 - val_loss: 4.0042 - val_accuracy: 0.9045
Epoch 7/15
12799/12799 [=========================

In [47]:
y_pred = model.predict_classes(X_test)
y_pred

array([[1],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]], dtype=int32)

In [48]:
print("-------------Confusion Matrix -----------------")
confusion_matrix(y_test, y_pred)

-------------Confusion Matrix -----------------


array([[2826,  281],
       [ 299, 2080]])

In [49]:
print("-------------Classification Report -----------------")
classification_report(y_test, y_pred)

-------------Classification Report -----------------


'              precision    recall  f1-score   support\n\n           0       0.90      0.91      0.91      3107\n           1       0.88      0.87      0.88      2379\n\n    accuracy                           0.89      5486\n   macro avg       0.89      0.89      0.89      5486\nweighted avg       0.89      0.89      0.89      5486\n'

In [50]:
model.save('LSTM_Bi_dropout.h5')